In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Part 3
</center>
</h1>
<div class=h1_cell>
<p>
In this section we will be exploring the random forest model in more detail. In particular, we will be experimenting with tuning our forest in search of better performance. This will require examining individual trees and their respective behavior and performance. 
</div>

In [77]:
import pandas as pd
import os

home_path = os.path.expanduser('~')

file_path = '/'

file_name = 'wrangled_game_table_trees.csv'

game_table = pd.read_csv(home_path + file_path + file_name)
pd.__version__

u'0.20.3'

In [78]:
pd.set_option('display.max_columns', None)

In [79]:
os.chdir(home_path + '/datascience_1')
!git pull

Already up to date.


In [80]:
import sys
sys.path.append(home_path + '/datascience_1')
from week7 import *
%who function

accuracy	 all_votes	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 
euclidean_distance	 f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 
k_fold	 k_means	 k_take_2	 oob_testing	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 
same_votes	 seeder	 tree_predictor	 vote_taker	 


In [81]:
game_table.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan,gls_binned,gls_low,gls_average,gls_high,gls_nan,forest1_oob,forest1_oob_types,forest2_oob,forest2_oob_types,forest3_oob,forest3_oob_types
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.000000,76.00,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0,high,0,0,1,0,1,true_positive,1,true_positive,1,true_positive
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2008.000000,82.00,1,average,0,1,0,0,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,2009.000000,80.00,1,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0,0,false_negative,1,true_positive,1,true_positive
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1,low,1,0,0,0,low,1,0,0,0,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive


<h1>
<center>
Tuning the Random Forest Model
</center>
</h1>
<div class=h1_cell>

<p>
We will start by looking back at our best random forest from Part 2.
<p>
This forest was our best performing from the last module. It had the best overall accuracy, f1, and informedness scores.
</div>

In [82]:
forest2_oob_types = game_table['forest2_oob_types'].value_counts()

print((accuracy(forest2_oob_types), f1(forest2_oob_types), informedness(forest2_oob_types)))

(0.75327471738740359, 0.012921751615218951, 0.005957167824620635)


<hr>
<h2>
Analyzing the Forest
</h2>
<p>
<div class=h1_cell>
<p>
We're going to analyze this forest based on the individual trees within it and try to improve it.
<p>
This next function takes a forest and compiles a matrix of all of the tree votes for the forest.
</div>

In [83]:
def all_votes(forest, table):
    result = []
    for i in range(len(table)):
        row_preds = []
        for x in range(len(forest)):
            row = table.loc[i]
            pred = tree_predictor(row, forest[x])
            row_preds.append(pred)
        result.append(row_preds)
    return result

In [84]:
forest_columns = [
    #North American Sales
    'nas_low', 'nas_average', 'nas_high',
    #European Sales
    'eus_low', 'eus_average', 'eus_high',
    #Japanese Sales
    'jps_low', 'jps_average', 'jps_high',
    #Other Sales
    'ots_low', 'ots_average', 'ots_high',
    #Global Sales
    'gls_low', 'gls_average', 'gls_high'
]


# I wrote this function for oob testing purposes
def oob_testing(forest, table):
    new_table = table.loc[forest[0]['oob']]
    
    for i in range(1, len(forest)):
        temp = table.loc[forest[i]['oob']]
        new_table = new_table.append(temp)
    return new_table

In [145]:
#Test on our forest
init_forest = forest_builder(game_table, forest_columns, 'rating_over_70', hypers={'total-trees':10})

oob_table = oob_testing(init_forest, game_table)
best_forest = forest_builder(oob_table, forest_columns, 'rating_over_70', hypers={'total-trees':10})

In [118]:
votes = all_votes(best_forest, game_table)

In [119]:
votes

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
 [0, 1, 1, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 1, 1, 1, 1, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0,

<div class=h1_cell>
<p>
This matrix will help us discover patterns and relationships among the trees in the forest. 
<p>
We will start with finding the number of rows with unanimous predictions.
</div>

In [30]:
def unanimous_rows(all_votes):
    unan_list = []
    for vote in all_votes:
        unan = True
        curr = vote[0]
        for i in vote:
            if i != curr:
                unan = False
                break
        if(unan):
            unan_list.append(vote)
    return unan_list

In [31]:
unan_rows = unanimous_rows(votes)

In [32]:
len(unan_rows)

16683

<div class=h1_cell>
<p>
There is a substantial number of unanimous rows in this forest. It seems like the majority of the trees produce unanimous rows of votes. I don't think this is a particularly useful attribute at the moment however.
</div>

<div class=h1_cell>
<p>
Now we're going to find which trees always produce the same vote.
</div>

In [21]:
def never_change(all_votes):
    constant_trees = []
    for i in range(len(all_votes[0])):
        same = True
        curr = all_votes[0][i]
        for j in range(len(all_votes)):
            if all_votes[j][i] != curr:
                same = False
                break
        if(same):
            constant_trees.append(i)
    return constant_trees

In [33]:
never = never_change(votes)

In [34]:
len(never)

0

<h1>
No luck with constants
</h1>
<div class=h1_cell>
<p>
Looks like there are no constant voters. We will have to continue analyzing the votes for another relationship we could use.
<p>
Let's try to find trees that always vote the same. If two trees vote the same they probably will not contribute much overall and we can get rid of them.
</div>

In [53]:
#creates all possible non equal combos of k
def k_take_2(k):
    result = []
    num_list = range(0, k)
    for i in range(len(num_list)):
        for j in range(len(num_list)):
            if (i != j):
                tup = (i, j)
                if tuple(reversed(tup)) not in result:  
                    result.append(tup)
    return result

In [115]:
def same_voters(all_votes):
    twins = []
    combos = k_take_2(len(all_votes[0]))
    for combo in combos:
        x = combo[0]
        y = combo[1]
        same = True
        for i in range(len(all_votes)):
            if (all_votes[i][x] != all_votes[i][y]):
                same = False
                break
        if (same):
            twins.append(combo)
    return twins


In [116]:
twins_test = same_voters([[0, 1, 0, 0, 0, 1],
                          [0, 0, 1, 1, 1, 0]])

twins_test

[(1, 5), (2, 3), (2, 4), (3, 4)]

<div class=h1_cell>
<p>
This algorithm I wrote for finding duplicate trees seems to work. Let's try it on our forest.
</div>

In [120]:
twin_trees = same_voters(votes)
twin_trees

[(1, 7), (3, 8)]

<h1>
Success!
</h1>
<div class=h1_cell>
<p>
Looks like there we found two pairs of trees who vote the same.
<p>
We can remove one of each tree.
</div>

In [146]:
# remove index 8
best_forest.pop(8)
# remove index 7
best_forest.pop(7)

len(best_forest)

8

<h1>
Test the new forest
</h1>
<div class=h1_cell>
<p>
We're going to see if removing these trees helped the performance of the forest.
</div>

In [156]:
new_oob_table = oob_testing(best_forest, game_table)
new_best_forest = forest_builder(new_oob_table, forest_columns, 'rating_over_70', hypers={'total-trees':8})

game_table['tuned_forest_oob'] = game_table.apply(lambda row: vote_taker(row, new_best_forest), axis=1)
game_table['tuned_forest_types'] = game_table.apply(lambda row: predictor_case(row, pred='tuned_forest_oob', target='rating_over_70'), axis=1)
tuned_forest_types = game_table['tuned_forest_types'].value_counts()
print(tuned_forest_types.sum())  # length of testing table
print((accuracy(tuned_forest_types), f1(tuned_forest_types), informedness(tuned_forest_types)))

16719
(0.75315509300795502, 0.011497005988023954, 0.0053129334412653773)


<div class=h1_cell>
<p>
Looks like we were right and the duplicates were not contributing positively. Removing them from our forest boosted our performance on all levels. However, considering these are random forests, the increase in performance could be attributed to the random factor. Just something to think about.
</div>

<h1>
Another New Method
</h1>
<div class=h1_cell>
<p>
Finally, we're going to try and find the worst performing tree and remove it from the forest. Getting rid of the worst sounds like it could only bennefit our performance.
</div>

In [137]:
def case_checker(row, pred, target):
    actual = row[target]
    prediction = pred
    if actual == 0 and prediction == 0:
        case = 'true_negative'
    elif actual == 1 and prediction == 1:
        case = 'true_positive'
    elif actual == 1 and prediction == 0:
        case = 'false_negative'
    else:
        case = 'false_positive'
    return case

In [141]:
def worst_tree(table, forest, target):
    totals = []
    for i in range(len(forest)):
        tree = forest[i]
        count = {i:0}
        for index, row in table.iterrows():
            pred = tree_predictor(row, tree)
            
            case = case_checker(row, pred, target)
            
            if(case == 'true_positive'):
                count[i] += 1
        totals.append(count)
    return totals

In [158]:
rf_totals = worst_tree(game_table, new_best_forest, 'rating_over_70')
rf_totals

[{0: 26}, {1: 4}, {2: 26}, {3: 27}, {4: 12}, {5: 3}, {6: 26}, {7: 28}]

<div class=h1_cell>
<p>
Looks like the tree at index 5 has the worst record for true_positives. We're going to get rid of it then.
</div>

In [159]:
new_best_forest.pop(5)
len(new_best_forest)

7

<div class=h1_cell>
<p>
Lets recalculate and see if there was any improvement.

</div>

In [160]:
new_oob_table2 = oob_testing(new_best_forest, game_table)
new_best_forest2 = forest_builder(new_oob_table2, forest_columns, 'rating_over_70', hypers={'total-trees':7})

game_table['tuned_forest_oob2'] = game_table.apply(lambda row: vote_taker(row, new_best_forest2), axis=1)
game_table['tuned_forest_types2'] = game_table.apply(lambda row: predictor_case(row, pred='tuned_forest_oob2', target='rating_over_70'), axis=1)
tuned_forest_types2 = game_table['tuned_forest_types2'].value_counts()

print((accuracy(tuned_forest_types2), f1(tuned_forest_types2), informedness(tuned_forest_types2)))

(0.75219809797236681, 0.001446131597975416, 0.00064423438335525773)


<div class=h1_cell>
<p>
Our performance seems to have gone down. Our f1 and informedness score seemed to have decreased rather significantly. Once again due to the random nature of the model it could explain why we see a decrease when we expected an increase. 
</div>

In [162]:
game_table.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,YoR_filled,kmeans_Critic_Score,rating_over_70,nas_binned,nas_low,nas_average,nas_high,nas_nan,eus_binned,eus_low,eus_average,eus_high,eus_nan,jps_binned,jps_low,jps_average,jps_high,jps_nan,ots_binned,ots_low,ots_average,ots_high,ots_nan,gls_binned,gls_low,gls_average,gls_high,gls_nan,forest1_oob,forest1_oob_types,forest2_oob,forest2_oob_types,forest3_oob,forest3_oob_types,tuned_forest_oob,tuned_forest_types,tuned_forest_oob2,tuned_forest_types2
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E,2006.000000,76.00,1,high,0,0,1,0,high,0,0,1,0,average,0,1,0,0,high,0,0,1,0,high,0,0,1,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,0,false_negative
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,0,false_negative
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2008.000000,82.00,1,average,0,1,0,0,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E,2009.000000,80.00,1,average,0,1,0,0,average,0,1,0,0,low,1,0,0,0,low,1,0,0,0,average,0,1,0,0,0,false_negative,1,true_positive,1,true_positive,0,false_negative,1,true_positive
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,2006.487356,86.55,1,low,1,0,0,0,low,1,0,0,0,high,0,0,1,0,low,1,0,0,0,average,0,1,0,0,1,true_positive,1,true_positive,1,true_positive,0,false_negative,0,false_negative


<h1>
<center>
Results
</center>
</h1>
<div class=h1_cell>
<p>
It seems like our efforts with duplicate voters was successful while our worst performing tree effort was not. For me, it is hard to tell wheter or not these results stem from the random nature of the model or directly from our efforts. Either way, I think it was an interesting experiment to dig deeper into this model.


In [163]:
import os

home_path =  os.path.expanduser('~')

file_path = '/'

file_name = 'wrangled_game_table_done.csv'

game_table.to_csv(home_path + file_path + file_name, index=False)